In [2]:
import numpy as np
from scipy import optimize as op
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import astropy.units as u
import astropy.cosmology.units as cu
from astropy.cosmology import WMAP9
from extinction import fm07 as fm
from dustmaps.sfd import SFDQuery
from astropy.coordinates import SkyCoord

In [3]:
#joseph has 22hnt D_lit as 83.98 before but now is 84, but still isn't published

In [4]:
peak_data = {'sn':['snztf18','sn18dfi','sn19rwd','sn20ano','sn20bio','sn20ikq','sn20rsc','sn20sbw','sn21gno','sn21heh','sn21pb','sn21vgn','sn22hnt','sn22jpx','sn22qzr'],
            'ra':['15h49m11.64s', '16h50m50.084s', '00h10m45.898s', '13h06m25.176s', '13h55m37.687s', '13h36m05.016s','01h19m56.503s','02h46m03.318s', '12h12m10.290s','07h59m47.290s','09h44m46.80s','16h21m10.510s','11h36m59.754s','10h10m10.00s','00h09m55.001s'],
            'dec':['32d17m16.68s', '45d23m52.44s', '21d08m20.73s', '53d28m45.53s', '40d28m39.15s', '28d59m00.11s', '38d11m09.66s', '03d19m47.66s', '13d14m57.05s', '25d21m20.99s', '51d41m14.6s', '36d03m40.36s',' 55d09m50.25s', '-11d04m50.05s', '-05d01m16.09s'],
            'mag_sce_g':[18.9, np.nan, 18.1, 19.1, 17.8, 18.3, 19.5, np.nan, 17.7, 17.5, 18.7, 18.3, 18.1, 16.3, 17.4],
            'mag_sce_r':[19.2, np.nan, 18.3, 19.6, 16.4, 18.5, 19.5, np.nan, 18.1, 17.4, 18.9, 18.3, 17.8, 16.4, 17.5],
            # 'mag_ni_g':[19.6, 18.5, 20.6, 17.1, 19.2, 21.7, 17.5, 18.5, 19.3, 18.7, 18.6, 17.1, 17.4],
            # 'mag_ni_r':[19.1, 18.1, 20.1, 17.0, 19.0, 20.8, 17.0, 17.7, 18.6, 18.3, 17.9, 16.9, 17.0],
            'z':[0.0582, 0.031, 0.017, 0.03113, 0.009, 0.0378, 0.0313, 0.023033, 0.006211, 0.026648, 0.03319, 0.03235, 0.0192, 0.015, 0.019],
            'd_lit':[263, np.nan, np.nan, np.nan, 29.9, np.nan, np.nan, np.nan, 30.5, 117.7, np.nan, np.nan, np.nan, np.nan, np.nan],
            'd_calc':[248.6, np.nan, 73.3, 133.8 , 38.9, 158.8, 134.5, np.nan, 26.8, 114.6, 141.7, 138.9, 82.7, 64.7, 81.9],
            'Mag_g_sce_lit':[-18.2, -17.841, np.nan, -16.65, np.nan, -18.10, -16.37, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
            'Mag_r_sce_lit':[-17.9, -17.646, np.nan, -16.19, np.nan, -17.86, -16.36, np.nan, np.nan, np.nan, -16.9, np.nan, np.nan, np.nan, np.nan],
            # 'Mag_g_ni':[-17.5, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
            # 'Mag_r_ni':[-18.0, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, -16.8, np.nan, np.nan, np.nan, np.nan],
            }
peak_df = pd.DataFrame(data=peak_data)

In [5]:
peak_df

,sn,ra,dec,mag_sce_g,mag_sce_r,z,d_lit,d_calc,Mag_g_sce_lit,Mag_r_sce_lit
0,snztf18,15h49m11.64s,32d17m16.68s,18.9,19.2,0.058200,263.0,248.6,-18.200,-17.900
1,sn18dfi,16h50m50.084s,45d23m52.44s,NaN,NaN,0.031000,NaN,NaN,-17.841,-17.646
2,sn19rwd,00h10m45.898s,21d08m20.73s,18.1,18.3,0.017000,NaN,73.3,NaN,NaN
3,sn20ano,13h06m25.176s,53d28m45.53s,19.1,19.6,0.031130,NaN,133.8,-16.650,-16.190
4,sn20bio,13h55m37.687s,40d28m39.15s,17.8,16.4,0.009000,29.9,38.9,NaN,NaN
5,sn20ikq,13h36m05.016s,28d59m00.11s,18.3,18.5,0.037800,NaN,158.8,-18.100,-17.860
6,sn20rsc,01h19m56.503s,38d11m09.66s,19.5,19.5,0.031300,NaN,134.5,-16.370,-16.360
7,sn20sbw,02h46m03.318s,03d19m47.66s,NaN,NaN,0.023033,NaN,NaN,NaN,NaN
8,sn21gno,12h12m10.290s,13d14m57.05s,17.7,18.1,0.006211,30.5,26.8,NaN,NaN
9,sn21heh,07h59m47.290s,25d21m20.99s,17.5,17.4,0.026648,117.7,114.6,NaN,NaN


In [6]:
#ho23 has 20ano Mag_g=-16.65 and Mag_r=-16.19   #matches my SCE peak estimate
#^^ has 20ikq Mag_g=-18.10 and Mag_r=-17.86     #matches my SCE peak estimate
#^^ has 20rsc Mag_g=-16.37 and Mag_r=-16.36     #matches my SCE peak estimate
#Rodríguez23 has 21heh Mag_r=-18.273            #unclear which peak is used for peak mag
#das23 has 21pb Mag_r_SCE=-16.9 and Mag_r_Ni=-16.8

In [7]:
z = peak_df.z.values * cu.redshift
peak_df['d_calc'] = np.array(z.to(u.Mpc, cu.redshift_distance(WMAP9, kind="comoving")))

In [8]:
peak_df

,sn,ra,dec,mag_sce_g,mag_sce_r,z,d_lit,d_calc,Mag_g_sce_lit,Mag_r_sce_lit
0,snztf18,15h49m11.64s,32d17m16.68s,18.9,19.2,0.058200,263.0,248.511363,-18.200,-17.900
1,sn18dfi,16h50m50.084s,45d23m52.44s,NaN,NaN,0.031000,NaN,133.167852,-17.841,-17.646
2,sn19rwd,00h10m45.898s,21d08m20.73s,18.1,18.3,0.017000,NaN,73.251209,NaN,NaN
3,sn20ano,13h06m25.176s,53d28m45.53s,19.1,19.6,0.031130,NaN,133.722483,-16.650,-16.190
4,sn20bio,13h55m37.687s,40d28m39.15s,17.8,16.4,0.009000,29.9,38.847391,NaN,NaN
5,sn20ikq,13h36m05.016s,28d59m00.11s,18.3,18.5,0.037800,NaN,162.136222,-18.100,-17.860
6,sn20rsc,01h19m56.503s,38d11m09.66s,19.5,19.5,0.031300,NaN,134.447721,-16.370,-16.360
7,sn20sbw,02h46m03.318s,03d19m47.66s,NaN,NaN,0.023033,NaN,99.116401,NaN,NaN
8,sn21gno,12h12m10.290s,13d14m57.05s,17.7,18.1,0.006211,30.5,26.825177,NaN,NaN
9,sn21heh,07h59m47.290s,25d21m20.99s,17.5,17.4,0.026648,117.7,114.582024,NaN,NaN


In [9]:
#correcting app mag measurements for MW dust extinction
wle={
    "g": 4830,
    "r": 6260,
}

for i in range(len(peak_data['sn'])):
    sfd = SFDQuery()
    coord = SkyCoord(peak_df.ra.iloc[i], peak_df.dec.iloc[i])

    exts = fm(
        np.asarray([wle[filt] * (1 + peak_df.loc[peak_df['sn']==peak_df.sn.iloc[i], 'z'].iloc[0]) for filt in wle.keys()]),
        sfd(coord),
    )
    #UPDATES DF APPARENT MAG COLUMNS TO BE MW CORRECTED
    peak_df.loc[peak_df['sn']==peak_df.sn.iloc[i], 'mag_sce_g'] -= exts[0]
    peak_df.loc[peak_df['sn']==peak_df.sn.iloc[i], 'mag_sce_r'] -= exts[1]

    #saving ext factor as column for export
    peak_df.loc[peak_df['sn']==peak_df.sn.iloc[i], 'ext_g'] = exts[0]
    peak_df.loc[peak_df['sn']==peak_df.sn.iloc[i], 'ext_r'] = exts[1]


In [10]:
#just printing the values for the paper table
for i in range(len(peak_data['sn'])):
    sfd = SFDQuery()
    coord = SkyCoord(peak_df.ra.iloc[i], peak_df.dec.iloc[i])

    exts = fm(
        np.asarray([wle[filt] * (1 + peak_df.loc[peak_df['sn']==peak_df.sn.iloc[i], 'z'].iloc[0]) for filt in wle.keys()]),
        sfd(coord),
    )
    print(peak_df['sn'].loc[peak_df['sn']==peak_df.sn.iloc[i]].iloc[0], exts[0], exts[1])

snztf18 0.024367652297056248 0.016695318138846362
sn18dfi 0.024894052314410695 0.01724317548445182
sn19rwd 0.08255261541164892 0.05751744407466776
sn20ano 0.024122809260505067 0.016708067268214707
sn20bio 0.01150459474717345 0.00804316343233838
sn20ikq 0.01343117936273243 0.009277361334584613
sn20rsc 0.05761919960439654 0.0399057125634598
sn20sbw 0.044073150137679956 0.030629263293419435
sn21gno 0.04168767522368923 0.029180103317268086
sn21heh 0.04162348222611881 0.028883129372752957
sn21pb 0.013012465521538424 0.009005114822292305
sn21vgn 0.01827471118849528 0.012651159391022546
sn22hnt 0.01768327344464704 0.012309105127626726
sn22jpx 0.05515072258652138 0.03845825277493543
sn22qzr 0.03576644090196691 0.02489867773798646


In [11]:
peak_df

,sn,ra,dec,mag_sce_g,mag_sce_r,z,d_lit,d_calc,Mag_g_sce_lit,Mag_r_sce_lit,ext_g,ext_r
0,snztf18,15h49m11.64s,32d17m16.68s,18.875632,19.183305,0.058200,263.0,248.511363,-18.200,-17.900,0.024368,0.016695
1,sn18dfi,16h50m50.084s,45d23m52.44s,NaN,NaN,0.031000,NaN,133.167852,-17.841,-17.646,0.024894,0.017243
2,sn19rwd,00h10m45.898s,21d08m20.73s,18.017447,18.242483,0.017000,NaN,73.251209,NaN,NaN,0.082553,0.057517
3,sn20ano,13h06m25.176s,53d28m45.53s,19.075877,19.583292,0.031130,NaN,133.722483,-16.650,-16.190,0.024123,0.016708
4,sn20bio,13h55m37.687s,40d28m39.15s,17.788495,16.391957,0.009000,29.9,38.847391,NaN,NaN,0.011505,0.008043
5,sn20ikq,13h36m05.016s,28d59m00.11s,18.286569,18.490723,0.037800,NaN,162.136222,-18.100,-17.860,0.013431,0.009277
6,sn20rsc,01h19m56.503s,38d11m09.66s,19.442381,19.460094,0.031300,NaN,134.447721,-16.370,-16.360,0.057619,0.039906
7,sn20sbw,02h46m03.318s,03d19m47.66s,NaN,NaN,0.023033,NaN,99.116401,NaN,NaN,0.044073,0.030629
8,sn21gno,12h12m10.290s,13d14m57.05s,17.658312,18.070820,0.006211,30.5,26.825177,NaN,NaN,0.041688,0.029180
9,sn21heh,07h59m47.290s,25d21m20.99s,17.458377,17.371117,0.026648,117.7,114.582024,NaN,NaN,0.041623,0.028883


In [12]:
def calc_abs_mag(df, mag_col, return_col):
    hubb_flow = (df['z']>=0.015)

    #calc abs mag if in hubble flow
    # M = m - 5log10(D/10pc) + 2.5log10(1+z)
    df.loc[(df.d_lit.isna())&(hubb_flow), return_col] =  (df.loc[(df.d_lit.isna())&(hubb_flow), mag_col] - 
                                                              5*np.log10((df.loc[(df.d_lit.isna())&(hubb_flow),'d_calc']*1e6) / 10) +
                                                              2.5*np.log10(1 + df.loc[(df.d_lit.isna())&(hubb_flow),'z']))
    
    df.loc[(df.d_lit.notna())&(hubb_flow), return_col] = (df.loc[(df.d_lit.notna())&(hubb_flow), mag_col] - 
                                                              5*np.log10((df.loc[(df.d_lit.notna())&(hubb_flow),'d_lit']*1e6) / 10) +
                                                              2.5*np.log10(1 + df.loc[(df.d_lit.notna())&(hubb_flow),'z']))
    
    #calc abs mag if NOT in hubble flow
    # M = m - 2.5log10(D/10pc)^2
    df.loc[(df.d_lit.isna())&(~hubb_flow), return_col] = (df.loc[(df.d_lit.isna())&(~hubb_flow), mag_col] - 
                                                              2.5*np.log10(((df.loc[(df.d_lit.isna())&(~hubb_flow), 'd_calc']*1e6)/10)**2))
    
    df.loc[(df.d_lit.notna())&(~hubb_flow), return_col] = (df.loc[(df.d_lit.notna())&(~hubb_flow), mag_col] - 
                                                               2.5*np.log10(((df.loc[(df.d_lit.notna())&(~hubb_flow), 'd_lit']*1e6)/10)**2))
    
    ######
    
    #calc conversion factor if in hubble flow
    df.loc[(df.d_lit.isna())&(hubb_flow), 'mag_conversion'] = (5*np.log10((df.loc[(df.d_lit.isna())&(hubb_flow),'d_calc']*1e6) / 10) + 
                                                                   2.5*np.log10(1 + df.loc[(df.d_lit.isna())&(hubb_flow),'z']))
    df.loc[(df.d_lit.notna())&(hubb_flow), 'mag_conversion'] = (5*np.log10((df.loc[(df.d_lit.notna())&(hubb_flow),'d_lit']*1e6) / 10) +
                                                                    2.5*np.log10(1 + df.loc[(df.d_lit.notna())&(hubb_flow),'z']))
    
    #calc conversion factor if NOT in hubble flow
    df.loc[(df.d_lit.isna())&(~hubb_flow), 'mag_conversion'] = 2.5*np.log10(((df.loc[(df.d_lit.isna())&(~hubb_flow), 'd_calc']*1e6)/10)**2)
    df.loc[(df.d_lit.notna())&(~hubb_flow), 'mag_conversion'] = 2.5*np.log10(((df.loc[(df.d_lit.notna())&(~hubb_flow), 'd_lit']*1e6)/10)**2)
    
    return

In [13]:
calc_abs_mag(peak_df, 'mag_sce_g', 'Mag_g_sce_calc')
calc_abs_mag(peak_df, 'mag_sce_r', 'Mag_r_sce_calc')

In [ ]:
peak_df[['sn','d_lit','d_calc']]

,sn,d_lit,d_calc
0,snztf18,263.0,248.511363
1,sn18dfi,NaN,133.167852
2,sn19rwd,NaN,73.251209
3,sn20ano,NaN,133.722483
4,sn20bio,29.9,38.847391
5,sn20ikq,NaN,162.136222
6,sn20rsc,NaN,134.447721
7,sn20sbw,NaN,99.116401
8,sn21gno,30.5,26.825177
9,sn21heh,117.7,114.582024


In [15]:
peak_df['Mag_g_sce_calc'].describe()

count    13.000000
mean    -16.767944
std       1.127423
min     -18.162727
25%     -17.722547
50%     -17.046742
75%     -16.288324
max     -14.589861
Name: Mag_g_sce_calc, dtype: float64

In [16]:
peak_df['Mag_r_sce_calc'].describe()

count    13.000000
mean    -16.742309
std       1.014657
min     -17.954212
25%     -17.518393
50%     -16.842735
75%     -16.063289
max     -14.350679
Name: Mag_r_sce_calc, dtype: float64

In [17]:
peak_df[['sn','ext_g','ext_r','mag_conversion']].to_csv('./data/mag_conversion_factors.csv', index=False)